In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [2]:
from typing import Generator, Optional

import datetime as dt
import os
import json

from constants import UNIBET_MIN_DATE

In [3]:
# Found in JS unibet code
class UnibetBetTRateType:
    SIMPLE_WINNER=1 # Mise de base: 1€<br>Trouvez le 1er cheval de l’arrivée.
    SIMPLE_PLACED=2 # "Mise de base: 1€<br>Trouvez 1 cheval parmi les 3 premiers sur les courses de 8 chevaux et plus OU 1 cheval parmi les 2 premiers sur les courses de 4 à 7 chevaux.
    JUMELE_WINNER=3 # Mise de base: 1€<br>Trouvez les 2 premiers chevaux dans l’ordre ou le désordre.
    JUMELE_ORDER=4 # Mise de base: 1€<br>Trouvez les 2 premiers chevaux dans l’ordre.
    JUMELE_PLACED=5 # Mise de base: 1€<br>Trouvez 2 chevaux parmi les 3 premiers à l’arrivée.
    TRIO=6 # Mise de base: 1€<br>Trouvez les 3 premiers chevaux.
    LEBOULET=7 # Mise de base: 1€<br>Trouvez le cheval qui arrive à la 4ème place.
    QUADRI=8 # Mise de base: 0.50€<br>Trouvez les 4 premiers chevaux, dans l’ordre ou le désordre.
    TRIO_ORDER=11 # Mise de base: 1€<br>Trouvez les 3 premiers chevaux dans l'ordre.
    FIVE_OVER_FIVE = 12 # Mise de base: 0.50€<br>Trouvez les 5 premiers chevaux, dans l’ordre ou le désordre.
    TWO_OVER_FOUR = 13 # Mise de base: 1€<br>Trouvez 2 chevaux parmi les 4 premiers à l’arrivée.
    DEUZIO=29 # Mise de base: 1€<br>Trouvez le cheval qui arrive à la 2ème place
    MIX_FOUR=15 # Mise de base : 1.50€<br>Mixez dans un même betslip: un Quadri + des Trios + des Jumelés Gagnants.
    MIX_FIVE = 16 # Mise de base: 2€<br>Mixez dans un même betslip: un 5 sur 5 + des Quadris + des Trios.
    MIX_S=31 # Mise de base: 3€<br>Mixez dans un même betslip: un Simple Gagnant + un Simple Placé + un Deuzio + un Boulet
    JUMELE=32 # Mise de base: 1€<br>Trouvez les 2 premiers chevaux à l’arrivée dans l’ordre ou le désordre.
    SIMPLE=33 # Mise de base: 1€<br>Trouvez le 1er cheval à l’arrivée gagnant ou placé.
    
    
class UnibetProbableType:
    morning_simple_gagnant_odds=5 #"matin cote" on simple_gagnant
    final_simple_gagnant_odds=6 # "cote directe" or "rapport_final" on simple_gagnant
    morning_simple_place_odds=7 # "matin cote" on simple_place, not sure
    final_simple_place_odds=8 # "cote directe" or "rapport_final" on simple_place not sure
    unknown_probable_type3=9
    final_deuzio_odds=13 # "rapport_final" on deuzio

In [4]:
UNIBET_DATA_PATH="./data/Unibet"

In [5]:
def date_countdown_generator(start_date:dt.date, end_date:Optional[dt.date])->Generator[dt.date, None, None]:
    end_date=end_date or dt.date.today()
    current_date= start_date
    while current_date <=end_date:
        yield current_date
        current_date+=dt.timedelta(days=1)

In [6]:
seen_runner_id =set()
def process_race(race:dict, complete_race:dict)->None:
    if complete_race.get('note')=='server error, no json':
        # Can not use complete_race
        return None
    race_name=race['name']
    assert complete_race['name']==race['name']
    
    race_datetime=dt.datetime.fromtimestamp(complete_race['starttime']/1000)
    if race_datetime.year <2020:
        return None
        
    if complete_race['betCounter'] is not None:
        print(complete_race['betCounter'], complete_race['friendlyUrl'])
        
    pronostic = complete_race['details']['pronostic']
    

In [7]:
def process_horse_show(horse_show:dict)->None:
    horse_show_datetime=dt.datetime.fromtimestamp(horse_show['date']/1000)
    unibet_n_horse_show=horse_show['rank']
    horse_show_ground=horse_show['ground']
    horse_show_place = horse_show['place']
    horse_show_country = horse_show['country']
    


In [8]:
for date in date_countdown_generator(start_date=dt.date(2020,1,1), end_date=dt.date(2020,4,1)):
    day_folder_path=os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" in os.listdir(day_folder_path):
        with open(os.path.join(day_folder_path, "programme.json"), 'r') as fp:
            programme=json.load(fp=fp)
        if 'data' not in programme:
            print(f'Can not import programme of {date.isoformat()}')
            continue
            
        horse_shows=programme['data']
        for horse_show in horse_shows:
            process_horse_show(horse_show=horse_show)
            
            if horse_show.get('races'):
                
                for race in horse_show['races']:
                    race_path = os.path.join(day_folder_path, f"R{horse_show['rank']}_C{race['rank']}.json")
                    with open(race_path, 'r') as fp:
                        complete_race=json.load(fp=fp)
                    process_race(race=race, complete_race=complete_race)
                    if set(complete_race['details']['probables'].keys())-{'13', '5', '6', '7', '8', '9'}:
                        print(complete_race['details']['probables'].keys(),complete_race['friendlyUrl'])

In [9]:
complete_race.keys()

dict_keys(['get_url', 'query_date', 'zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [11]:
horse_show

{'zeturfId': 40699,
 'date': 1585756800545,
 'going': 0.0,
 'place': 'BERGSAKER',
 'country': 'SUÈDE',
 'rank': 5,
 'ground': '-',
 'video': 0,
 'isActive': False,
 'betCounter': None,
 'races': [{'zeturfId': 250580,
   'meetingId': 40699,
   'starttime': 1585758420000,
   'date': '2020-04-01',
   'type': 'Attelé',
   'name': 'Prix Ukrainienne',
   'rank': 1,
   'conditions': "3 ans - Départ à l'Autostart - Sable - Corde à Gauche",
   'stake': 12334,
   'arjelLevel': '2',
   'distance': 2140,
   'betCounter': None,
   'friendlyUrl': '/turf/race/01-04-2020-R5-C1-bergsaker-prix-ukrainienne.html',
   'runners': [{'zeturfId': 3058041,
     'raceId': 250580,
     'rank': 1,
     'name': 'HAZELNUT SISU (SE)',
     'jockey': 'Forss Mik.',
     'weight': 0,
     'distance': 2140,
     'status': '40',
     'team': 0,
     'draw': 0,
     'blinkers': '0',
     'shoes': '',
     'position': '3',
     'silk': '0662_202004220_0707.gif',
     'betcounter': 0,
     'details': None},
    {'zeturfId': 

In [14]:
complete_race['details']['pronostic']

'<strong>Arnie Express sur la voie du succès !</strong><br /><br />Support du <strong>5 sur 5, le pari toujours spéculatif d\'Unibet</strong>, ce Prix Uraki des Sarts mettra en scène douze trotteurs âgés de quatre à neuf ans qui en découdront sur le parcours des 1.640 mètres de l\'hippodrome de Bergsaker avec un départ donné à l\'aide de l\'autostart. Double lauréat sur les 1.640 mètres de Solvalla au cours du mois de février, <strong>Arnie Express</strong> vient d\'être battu de peu pour un bon élément du redoutable Svanthe Bäth. Idéalement engagé au plafond des gains et bien placé au départ, ce protégé de Pekka Korpi s\'annonce redoutable. <strong>Captain My Captain</strong>, récent deuxième sur ce parcours en 1\'12"3 alors qu\'il s\'élançait du numéro 10 derrière la voiture, et <strong>Variety Show</strong>, des plus réguliers depuis le début de l\'année, ne partiront pas battus d\'avance, loin s\'en faut. Attention à <strong>Ready Lane</strong>, capable de bonnes choses quand il ne

In [25]:
complete_race['friendlyUrl']

'/turf/race/01-04-2020-R5-C9-bergsaker-prix-uraki-des-sarts.html'

In [26]:
complete_race['details'].keys()

dict_keys(['zeturfId', 'betList', 'pronostic', 'pronosticList', 'raceDuration', 'betState', 'photos', 'raceBetStatus', 'probables', 'rates'])

In [27]:
set(complete_race['details']['probables'].keys())

{'13', '5', '6', '7', '8', '9'}

In [28]:
complete_race['details']['rates']

[{'type': '1', 'combination': '4', 'rate': '1.9'},
 {'type': '2', 'combination': '4', 'rate': '1.1'},
 {'type': '2', 'combination': '6', 'rate': '3.2'},
 {'type': '2', 'combination': '7', 'rate': '1.8'},
 {'type': '3', 'combination': '4-6', 'rate': '7.9'},
 {'type': '5', 'combination': '4-6', 'rate': '4.3'},
 {'type': '5', 'combination': '4-7', 'rate': '4.2'},
 {'type': '5', 'combination': '6-7', 'rate': '7.6'},
 {'type': '6', 'combination': '4-6-7', 'rate': '31.3'},
 {'type': '13', 'combination': '4-6', 'rate': '2.8'},
 {'type': '13', 'combination': '4-7', 'rate': '2.8'},
 {'type': '13', 'combination': '4-3', 'rate': '2.8'},
 {'type': '13', 'combination': '6-7', 'rate': '2.8'},
 {'type': '13', 'combination': '6-3', 'rate': '2.8'},
 {'type': '13', 'combination': '7-3', 'rate': '2.8'},
 {'type': '8', 'combination': '4-6-7-3', 'rate': '67.2'},
 {'type': '12', 'combination': '4-6-7-3-9', 'rate': '98.1'},
 {'type': '29', 'combination': '6', 'rate': '14.7'},
 {'type': '7', 'combination': '3

In [20]:
complete_race['details']['pronosticList']

[{'rank': 1,
  'number': 1,
  'name': 'DALARNA (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 2,
  'number': 4,
  'name': 'HIGH NOON RIDER (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 3,
  'number': 2,
  'name': 'FLOWMOTION (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 4,
  'number': 10,
  'name': 'JAKARTA (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 5,
  'number': 3,
  'name': 'CELEBRATION (GB)',
  'comment': None,
  'long_comment': None},
 {'rank': 6,
  'number': 7,
  'name': 'TIL THE END (US)',
  'comment': None,
  'long_comment': None}]

In [13]:
complete_race['details']['betList']

[{'type': 1,
  'rate': 86.5,
  'max': 450,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 2,
  'rate': 82.0,
  'max': 450,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 3,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 5,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 4,
  'rate': 74.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 6,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 11,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 13,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 8,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status

In [52]:
complete_race['results']

{'zeturfId': 252428,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [4]},
  {'position': 2, 'numbers': [1]},
  {'position': 3, 'numbers': [5]},
  {'position': 4, 'numbers': [2]},
  {'position': 5, 'numbers': [6]}],
 'liveComments': []}

In [47]:
race['meetingId']

138

In [48]:
complete_race

{'get_url': 'https://www.unibet.fr/zones/turf/race.json?raceId=1003',
 'query_date': '2020-04-30T22:12:42.465135',
 'note': 'server error, no json'}

In [28]:
race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [42]:
race['results']

{'zeturfId': 2109,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [15]},
  {'position': 2, 'numbers': [4]},
  {'position': 3, 'numbers': [8]},
  {'position': 4, 'numbers': [6]},
  {'position': 5, 'numbers': [12]},
  {'position': 6, 'numbers': [14]},
  {'position': 7, 'numbers': [13]}],
 'liveComments': []}

In [41]:
race['betCounter']

In [34]:
print(race['meetingId'])

321

In [36]:
race['date']

'2005-08-22'

In [29]:
race['conditions']

"Départ à l'Autostart - Pour poulains entiers et hongres de 4 ans, n'ayant pas gagné 49.000 &euro; - Grande piste"

In [32]:
dt.datetime.fromtimestamp(race['starttime']/1000)

datetime.datetime(2005, 8, 22, 13, 50)

In [35]:
race['details']

{'zeturfId': 2109,
 'betList': [{'type': 1,
   'rate': 95.0,
   'max': 250,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 2,
   'rate': 80.0,
   'max': 250,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 3,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 5,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 6,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'}],
 'pronostic': None,
 'pronosticList': None,
 'raceDuration': '2\'33"97',
 'betState': 16,
 'photos': None,
 'raceBetStatus': 'PAID',
 'probables': None,
 'rates': None}

In [43]:
horse_show['rank']

1

In [44]:
race['rank']

1

In [48]:
with open(race_path, 'r') as fp:
    complete_race=json.load(fp=fp)

In [53]:
del complete_race['get_url'], complete_race['query_date']

In [51]:
race

{'zeturfId': 2109,
 'meetingId': 321,
 'starttime': 1124711400000,
 'date': '2005-08-22',
 'type': 'Attelé',
 'name': 'Prix de Jussey',
 'rank': 1,
 'conditions': "Départ à l'Autostart - Pour poulains entiers et hongres de 4 ans, n'ayant pas gagné 49.000 &euro; - Grande piste",
 'stake': 36000,
 'arjelLevel': '2',
 'distance': 2100,
 'betCounter': None,
 'friendlyUrl': '/turf/race/22-08-2005-R1-C1-vincennes-prix-de-jussey.html',
 'runners': [{'zeturfId': 27037,
   'raceId': 2109,
   'rank': 16,
   'name': 'NEVER DE LIVET (FR)',
   'jockey': 'Verbeeck J.',
   'weight': 0,
   'distance': 2100,
   'status': '40',
   'team': 0,
   'draw': 0,
   'blinkers': '0',
   'shoes': '',
   'position': '0',
   'silk': None,
   'betcounter': 0,
   'details': None},
  {'zeturfId': 27038,
   'raceId': 2109,
   'rank': 15,
   'name': 'NENUPHAR (FR)',
   'jockey': 'Levesque P.',
   'weight': 0,
   'distance': 2100,
   'status': '40',
   'team': 0,
   'draw': 0,
   'blinkers': '0',
   'shoes': '4',
   'pos

In [57]:
complete_race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [58]:
race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [60]:
shared_items = {k: complete_race[k] for k in complete_race if k in race and complete_race[k] == race[k]}
print(len(shared_items))

14


In [61]:
len(race)

16

In [63]:
difference_items = {k for k in complete_race if k in race and complete_race[k] != race[k]}

In [64]:
difference_items

{'details', 'runners'}

In [75]:
complete_race['runners']

[{'zeturfId': 27052,
  'raceId': 2109,
  'rank': 1,
  'name': 'NUMERO PRIVE (FR)',
  'jockey': 'Lannoo G.',
  'weight': 0,
  'distance': 2100,
  'status': '40',
  'team': 0,
  'draw': 0,
  'blinkers': '0',
  'shoes': '',
  'position': '0',
  'silk': None,
  'betcounter': 0,
  'details': {'zeturfId': 27052,
   'stakes': 41950,
   'musique': '1a 5a 3a 5a 1a 5a (04) 0a',
   'sex': 'M',
   'age': '6',
   'owner': 'M Bernard Marie Dubois',
   'trainer': 'Hojka B.',
   'coat': 'B.',
   'origins': "Cygnus d'Odyssee-Hollywood Melody",
   'comment': "Vient de remporter sa course à Enghien sur un parcours de vitesse. Se heurte à une opposition plus relevée. Pas hors d'affaire pour les places à l'issue d'un parcours caché.",
   'time': None,
   'reduction': None,
   'length': None}},
 {'zeturfId': 27051,
  'raceId': 2109,
  'rank': 2,
  'name': 'NICE MORNING (FR)',
  'jockey': 'Gilbert Martens',
  'weight': 0,
  'distance': 2100,
  'status': '40',
  'team': 0,
  'draw': 0,
  'blinkers': '0',
  's

In [69]:
race['runners'][0].keys()

dict_keys(['zeturfId', 'raceId', 'rank', 'name', 'jockey', 'weight', 'distance', 'status', 'team', 'draw', 'blinkers', 'shoes', 'position', 'silk', 'betcounter', 'details'])

In [70]:
x=complete_race['runners'][0]
y=race['runners'][0]
{k for k in x if k in y and x[k] != y[k]}

{'details', 'jockey', 'name', 'rank', 'zeturfId'}

In [78]:
race['runners'][0]

{'zeturfId': 27037,
 'raceId': 2109,
 'rank': 16,
 'name': 'NEVER DE LIVET (FR)',
 'jockey': 'Verbeeck J.',
 'weight': 0,
 'distance': 2100,
 'status': '40',
 'team': 0,
 'draw': 0,
 'blinkers': '0',
 'shoes': '',
 'position': '0',
 'silk': None,
 'betcounter': 0,
 'details': None}

In [76]:
for i, runner in enumerate(complete_race['runners']):
    print(i, runner['name'])

0 NUMERO PRIVE (FR)
1 NICE MORNING (FR)
2 NEW PISTOL (FR)
3 NEOPS RIVELLIERE (FR)
4 NICO D'AUSSY (FR)
5 NITBY (FR)
6 NEW LOOK DE CANISY (FR)
7 NIARPAN (FR)
8 NICCOLO D'AMOUR (FR)
9 NOVELINO (FR)
10 NESS MARCEAUX (FR)
11 NOBLE ANGEVIN (FR)
12 NEW HOPE (FR)
13 NO COMMENTARY (FR)
14 NENUPHAR (FR)
15 NEVER DE LIVET (FR)


In [79]:
complete_race['runners'][15]

{'zeturfId': 27037,
 'raceId': 2109,
 'rank': 16,
 'name': 'NEVER DE LIVET (FR)',
 'jockey': 'Verbeeck J.',
 'weight': 0,
 'distance': 2100,
 'status': '40',
 'team': 0,
 'draw': 0,
 'blinkers': '0',
 'shoes': '',
 'position': '0',
 'silk': None,
 'betcounter': 0,
 'details': {'zeturfId': 27037,
  'stakes': 35870,
  'musique': '5a 8a 2a Dm 3a (04) 2a Da',
  'sex': 'H',
  'age': '9',
  'owner': 'Mlle Caroline Cheradame',
  'trainer': 'Cheradame Mlle C.',
  'coat': 'Al.',
  'origins': 'Carpe Diem-vassia Pellois',
  'comment': "Possède un brin de qualité, mais risque encore de manquer  de compétition à ce niveau, venant juste d'effectuer sa rentrée sur l'herbe de Carentan. Malgré le talent de son driver, on se contentera de le regarder courir.",
  'time': None,
  'reduction': None,
  'length': None}}

In [81]:
complete_race['results']

{'zeturfId': 2109,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [15]},
  {'position': 2, 'numbers': [4]},
  {'position': 3, 'numbers': [8]},
  {'position': 4, 'numbers': [6]},
  {'position': 5, 'numbers': [12]},
  {'position': 6, 'numbers': [14]},
  {'position': 7, 'numbers': [13]}],
 'liveComments': []}

In [82]:
for runner in complete_race['runners']:
    print(runner['rank'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
